In [1]:
import pandas as pd
import numpy as np
import json
# plotting
import matplotlib.pyplot as plt
#opening data
import os
import pathlib
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [4]:
#Opening Dataset

path_players = os.path.join(str(pathlib.Path().resolve()),  'data', 'Wyscout', 'teams.json')
path_event = os.path.join(str(pathlib.Path().resolve()),  'data', 'Wyscout', 'events_England.json')

with open(path_players) as f:
    teams = json.load(f)
teams_df = pd.DataFrame(teams)

with open(path_event) as f:
    event = json.load(f)
train = pd.DataFrame(event)

teams_df = teams_df.rename(columns={"wyId":"teamId"})


In [5]:
#get corners
corners = train.loc[train["subEventName"] == "Corner"]
#count corners by team
corners_by_team = corners.groupby(['teamId']).size().reset_index(name='counts')
#merge with team name
summary = corners_by_team.merge(teams_df[["name", "teamId"]], how = "left", on = ["teamId"])
#count corners by team by game
corners_by_game = corners.groupby(['teamId', "matchId"]).size().reset_index(name='counts')
#merge with team name
summary2 = corners_by_game.merge(teams_df[["name", "teamId"]], how = "left", on = ["teamId"])

In [6]:
from statsmodels.stats.weightstats import ztest

#get city corners
city_corners = summary2.loc[summary2["name"] == 'Manchester City']["counts"]

#test
t, pvalue = ztest(city_corners,  value=8)
#checking outcome
if pvalue < 0.05:
    print("P-value amounts to", pvalue, "- We reject null hypothesis - Manchester City do not take 8 corners per game")
else:
    print("P-value amounts to", pvalue, " - We do not reject null hypothesis - Manchester City take 8 corners per game")

P-value amounts to 0.34703298713007624  - We do not reject null hypothesis - Manchester City take 8 corners per game


In [15]:
t, pvalue = ztest(city_corners,  value=6, alternative = "larger")
if pvalue < 0.05:
    print("P-value amounts to", pvalue, "- We reject null hypothesis - Manchester City take more than 6 corners per game")
else:
    print("P-value amounts to", pvalue, " - We do not reject null hypothesis - Manchester City do not take 6 more corners per game")

P-value amounts to 0.0023931156479123942 - We reject null hypothesis - Manchester City take more than 6 corners per game


In [16]:
mean = summary["counts"].mean()
std = summary["counts"].std()


from scipy.stats import ttest_1samp
leicester_corners = summary.loc[summary["name"] == "Leicester City"]["counts"].iloc[0]
t, pvalue = ttest_1samp(summary["counts"], leicester_corners)

if pvalue < 0.05:
    print("P-value amounts to", pvalue, "- We reject null hypothesis - Leicester City do not take average number of corners than league average")
else:
    print("P-value amounts to", pvalue, " - We do not reject null hypothesis - Leicester City take average number of corners than league average")

P-value amounts to 0.4023279517451914  - We do not reject null hypothesis - Leicester City take average number of corners than league average


In [17]:
from scipy.stats import ttest_1samp
arsenal_corners = summary.loc[summary["name"] == "Arsenal"]["counts"].iloc[0]
t, pvalue = ttest_1samp(summary["counts"], arsenal_corners, alternative='less')

if pvalue < 0.05:
    print("P-value amounts to", pvalue, "- We reject null hypothesis - Arsenal take more corners than league average")
else:
    print("P-value amounts to", pvalue, " - We do not reject null hypothesis - Arsenal do not take more corners than league average")

P-value amounts to 0.0016098690970901368 - We reject null hypothesis - Arsenal take more corners than league average


In [18]:
#check if united takes the same average number of corners per game as liverpool
liverpool_corners = summary2.loc[summary2["name"] == 'Liverpool']["counts"]
united_corners = summary2.loc[summary2["name"] == 'Manchester United']["counts"]

from scipy.stats import ttest_ind
t, pvalue  = ttest_ind(a=liverpool_corners, b=united_corners, equal_var=True)

if pvalue < 0.05:
    print("P-value amounts to", pvalue, "- We reject null hypothesis - Liverpool took different number of corners per game than United")
else:
    print("P-value amounts to", pvalue, " - We do not reject null hypothesis - Liverpool took the same number of corners per game as United")

P-value amounts to 0.5879909398542313  - We do not reject null hypothesis - Liverpool took the same number of corners per game as United


In [19]:
city_corners = summary2.loc[summary2["name"] == 'Manchester City']["counts"]
castle_corners = summary2.loc[summary2["name"] == 'Newcastle United']["counts"]

from scipy.stats import ttest_ind
t, pvalue  = ttest_ind(a=city_corners, b=castle_corners, equal_var=True, alternative = "greater")

if pvalue < 0.05:
    print("P-value amounts to", pvalue, "- We reject null hypothesis - City took more corners per game than Newcastle")
else:
    print("P-value amounts to", pvalue, " - We do not reject null hypothesis - City did not  take the more corners per game than Newcastle")

P-value amounts to 1.4280208353516603e-05 - We reject null hypothesis - City took more corners per game than Newcastle
